In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [118]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
from utilities.misc import get_qubits_involved, reindex_symbol, shift_symbols_down
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.misc import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter
from utilities.evaluator import Evaluator
from utilities.gate_killer import GateKiller
from ast import literal_eval


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
np.random.seed(0)
translator = CirqTranslater(3, untouchable_blocks = [1])
translator.env_qubits = [2]
translator.encoder_id = 0
translator.channel_id = 1
translator.decoder_id = 2
simplifier = Simplifier(translator)
etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)
killer = GateKiller(translator, mode="discrimination", params = etas)
inserter = IdInserter(translator.n_qubits, untouchable_blocks=translator.channel_id)
args_evaluator = {"n_qubits":translator.n_qubits, "problem":"acd","params":etas}
evaluator = Evaluator(args=args_evaluator, lower_bound_cost=minimizer.lower_bound_cost, nrun=0)

In [120]:
cdb = []
for ind, qubits in list(translator.indexed_cnots.items()):
    cdb.append(gate_template(int(ind), block_id=0))
    con, tar = qubits
    cdb.append(gate_template(int(con + translator.number_of_cnots), param_value = np.random.random()))
    cdb.append(gate_template(int(tar + translator.number_of_cnots + translator.n_qubits), param_value = np.random.random()))
circuit_db = pd.DataFrame(cdb)


In [121]:
circuit, circuit_db = translator.give_circuit(circuit_db)

In [122]:
circuit

(0, 0): ───@───Rz(th_0)───@───Rz(th_2)───X───Rx(th_5)──────────────────X───Rx(th_9)───────────────────
           │              │              │                             │
(0, 1): ───X───Rx(th_1)───┼──────────────@───Rz(th_4)───@───Rz(th_6)───┼──────────────X───Rx(th_11)───
                          │                             │              │              │
(0, 2): ──────────────────X───Rx(th_3)──────────────────X───Rx(th_7)───@───Rz(th_8)───@───Rz(th_10)───

In [123]:
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f593d4dfac8>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f593d4dfac8>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f593d4dfac8>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f593d4dfac8>>
<bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7f593d4dfac8>>


In [124]:
gates_on_qubit, on_qubit_order = simplifier.get_positional_dbs(circuit, circuit_db)
simplified_db = circuit_db.copy()

In [125]:
mdif = max_diff(translator , circuit_db, simplified_db)
mdif

0.0